In [1]:
# Подключение к Google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install flask-ngrok

In [3]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [4]:
# Пробный запуск Flask

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/a")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://865bceae6ebd.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/Dec/2020 18:13:33] "GET /a HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2020 18:13:34] "GET /favicon.ico HTTP/1.1" 404 -


In [5]:
!pip install h2o

     |████████████████████████████████| 164.6MB 82kB/s 
  Created wheel for h2o: filename=h2o-3.32.0.2-py2.py3-none-any.whl size=164620456 sha256=9b2701d7ceafe8254525a536c2c538bb744a6315928b5f898b2dbf80cd40b326
  Stored in directory: /root/.cache/pip/wheels/42/bd/ea/218fd15724eddf6fa7fc8fab802b6fa592e623d87199679721
Successfully built h2o


In [6]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.9.1" 2020-11-04; OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp54olyg8p
  JVM stdout: /tmp/tmp54olyg8p/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp54olyg8p/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.2
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_unknownUser_02pudv
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [7]:
# Загружаем обученные модели 
model_glm_binomial = h2o.load_model('/content/drive/My Drive/my_time_rows/GLM_model_python_1607029343503_1')


In [9]:
# Запуск Flask

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

def map_for_dict_Gender(Gender):
  dict_Gender = {'Male':0, 'Female':1}
  res = dict_Gender.get(Gender)
  return res

def map_for_dict_MariStat(MariStat):
  dict_MariStat = {'Other':0, 'Alone':1}
  res = dict_MariStat.get(MariStat)
  return res

def f_VehUsage_Professional(VehUsage):
  if VehUsage == 'Professional':
    VehUsage_Professional = 1
  else:
    VehUsage_Professional = 0
  return(VehUsage_Professional)

def f_VehUsage_Private_trip_to_office(VehUsage):
  if VehUsage == 'Private+trip to office':
    VehUsage_Private_trip_to_office = 1
  else:
    VehUsage_Private_trip_to_office = 0
  return(VehUsage_Private_trip_to_office)

def f_VehUsage_Private(VehUsage):
  if VehUsage == 'Private':
    VehUsage_Private = 1
  else:
    VehUsage_Privatel = 0
  return(VehUsage_Private)

def f_VehUsage_Professional_run(VehUsage):
  if VehUsage == 'Professional run':
    VehUsage_Professional_run = 1
  else:
    VehUsage_Professional_run = 0
  return(VehUsage_Professional_run)

def return_NewH2o_Frame():
  columns = [
      'LicAge',
      'Gender',
      'MariStat',
      'DrivAge',
      'HasKmLimit',
      'BonusMalus',
      'OutUseNb',
      'RiskArea',
      'VehUsg_Private',
      'VehUsg_Private+trip to office',
      'VehUsg_Professional',
      'VehUsg_Professional run',
      'CSP1',
      'CSP2',
      'CSP3',
      'CSP6',
      'CSP7',
      'CSP20',
      'CSP21',
      'CSP22',
      'CSP26',
      'CSP37',
      'CSP40',
      'CSP42',
      'CSP46',
      'CSP47',
      'CSP48',
      'CSP49',
      'CSP50',
      'CSP55',
      'CSP56',
      'CSP57',
      'CSP60',
      'CSP65',
      'CSP66'
      ]
  return h2o.H2OFrame([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]], column_names = columns)

@app.route("/")
def index():
    return "API for predict service"

@app.route('/predict', methods=['GET', 'POST'])
def predict3():

    try:

      json_input = request.json
      
      print(json_input)
      ID = json_input["ID"]
      LicAge = json_input["LicAge"]
      Gender = map_for_dict_Gender(json_input["Gender"])
      MariStat = map_for_dict_MariStat(json_input["MariStat"])
      DrivAge = json_input["DrivAge"]
      HasKmLimit = json_input["HasKmLimit"]
      BonusMalus = json_input["BonusMalus"]
      OutUseNb = json_input["OutUseNb"]
      RiskArea = json_input["RiskArea"]
      VehUsg_Private = f_VehUsage_Private(json_input["VehUsage"])
      VehUsg_Private_trip_to_office = f_VehUsage_Private_trip_to_office(json_input["VehUsage"])
      VehUsg_Professional = f_VehUsage_Professional(json_input["VehUsage"])
      VehUsg_Professional_run = f_VehUsage_Professional_run(json_input["VehUsage"])
      CSP1 = 0
      CSP2 = 0
      CSP3 = 0
      CSP6 = 0
      CSP7 = 0
      CSP20 = 0
      CSP21 = 0
      CSP22 = 0
      CSP26 = 0
      CSP37 = 0
      CSP40 = 0
      CSP42 = 0
      CSP46 = 0
      CSP47 = 0
      CSP48 = 0
      CSP49 = 0
      CSP50 = 0
      CSP55 = 0
      CSP56 = 0
      CSP57 = 0
      CSP60 = 0
      CSP65 = 0
      CSP66 = 0

      hf = return_NewH2o_Frame()

      hf[0, 'LicAge'] = LicAge
      hf[0, 'Gender'] = Gender
      hf[0, 'MariStat'] = MariStat
      hf[0, 'DrivAge'] = DrivAge
      hf[0, 'HasKmLimit'] = HasKmLimit
      hf[0, 'BonusMalus'] = BonusMalus
      hf[0, 'OutUseNb'] = OutUseNb
      hf[0, 'RiskArea'] = RiskArea
      hf[0, 'VehUsg_Private'] = VehUsg_Private
      hf[0, 'VehUsg_Private+trip to office'] = VehUsg_Private_trip_to_office
      hf[0, 'VehUsg_Professional'] = VehUsg_Professional
      hf[0, 'VehUsg_Professional run'] = VehUsg_Professional_run
      hf[0, 'CSP1'] = CSP1
      hf[0, 'CSP2'] = CSP2
      hf[0, 'CSP3'] = CSP3
      hf[0, 'CSP6'] = CSP6
      hf[0, 'CSP7'] = CSP7
      hf[0, 'CSP20'] = CSP20
      hf[0, 'CSP21'] = CSP21
      hf[0, 'CSP22'] = CSP22
      hf[0, 'CSP26'] = CSP26
      hf[0, 'CSP37'] = CSP37
      hf[0, 'CSP40'] = CSP40
      hf[0, 'CSP42'] = CSP42
      hf[0, 'CSP46'] = CSP46
      hf[0, 'CSP47'] = CSP47
      hf[0, 'CSP48'] = CSP48
      hf[0, 'CSP49'] = CSP49
      hf[0, 'CSP50'] = CSP50
      hf[0, 'CSP55'] = CSP55
      hf[0, 'CSP56'] = CSP56
      hf[0, 'CSP57'] = CSP57
      hf[0, 'CSP60'] = CSP60
      hf[0, 'CSP65'] = CSP65
      hf[0, 'CSP66'] = CSP66


      prediction_Binomial = model_glm_binomial.predict(hf)
      value_Binomial  = prediction_Binomial.as_data_frame()['predict'][0]/1

      return jsonify({'ID': json_input['ID'], 'value_Binomial':value_Binomial})

    except:
      
      return "Error"

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://938c354e006b.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{'ID': 1, 'LicAge': 468, 'RecordBeg': '2004-01-01', 'RecordEnd': '', 'VehAge': '', 'Gender': 'Male', 'MariStat': 'Other', 'SocioCateg': 'CSP50', 'VehUsage': 'Private', 'DrivAge': 67, 'HasKmLimit': 0, 'BonusMalus': 50, 'OutUseNb': 0, 'RiskArea': 0}
Parse progress: |█████████████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'driver_minage_m': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'driver_minage_f': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'driver_minage_m_2': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'driver_minage_f_2': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'VehUsage_Private': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missin